In [1]:
import os
os.chdir("/usr/src/app")

In [2]:

from deepsetstats.paths import (
    PATH_VIDEOS,
    PATH_MASTER_GS,
    PATH_MASTER_TENNIS_TV,
    PATH_MASTER_VIDEOS
)
import pandas as pd
import numpy as np
from deepsetstats.dataset.court_detection.utils import Utils

In [11]:
# Set the maximum number of rows to display
pd.set_option('display.max_rows', 70)

In [3]:
dfm = pd.read_parquet(PATH_MASTER_VIDEOS, engine="pyarrow")

In [21]:
dfm

,video_id,tournament_id,tournament_name,title,player_id,is_downloaded
3364,kPbwy5S8Qu0,0,Australian Open,Rafael Nadal v Mackenzie McDonald Extended Hig...,"[59, 138]",True
3138,hKm4ozoVyeM,0,Australian Open,Diego Schwartzman v Christopher O'Connell High...,"[73, 94]",False
1396,J3b_vrS8FiQ,0,Australian Open,Elena Rybakina v Jelena Ostapenko Condensed Ma...,[8451],True
3141,hNUvriAVxHk,0,Australian Open,Tallon Griekspoor v Pavel Kotov Highlights | A...,"[36, 86]",True
3933,sMWoplKZK3o,0,Australian Open,Matos/Vega Hernandez v Nys/Zielinski Highlight...,[4115],True
...,...,...,...,...,...,...
4109,v5EsxAYcHVs,64,ATP Finals,Melzer/Roger-Vasselin vs Koolhof/Mektic | Nitt...,[3178],False
2177,U4U4Oqkoaic,64,ATP Finals,Goffin and Dimitrov triumph; SF line-up set | ...,"[19, 99]",False
154,1M7N0-S2PjY,64,ATP Finals,Rafael Nadal vs Stefanos Tsitsipas: Extended H...,"[4, 138]",False
1897,Q3IsKsTFQwY,64,ATP Finals,DRAMATIC FINALE: Stefanos Tsitsipas Faces Dani...,"[2, 4]",True


### Videos downloaded per tournament

In [17]:
cross_tab = pd.crosstab(index=dfm['tournament_id'], columns=dfm['is_downloaded'])
total_true = cross_tab[True].sum()
total_false = cross_tab[False].sum()

# Create a new column 'TrueRatio' with the ratio of True counts
cross_tab['coverage(%)'] = np.round(cross_tab[True]*100 / (cross_tab[True] + cross_tab[False]),2)

In [18]:
cross_tab.sort_values("tournament_id").head(65)

is_downloaded,False,True,coverage(%)
tournament_id,,,
0,211,430,67.08
1,0,343,100.00
2,32,420,92.92
3,490,59,10.75
4,89,44,33.08
5,79,65,45.14
6,48,42,46.67
7,51,58,53.21
8,55,64,53.78


### Videos per tournament

In [19]:
dfm.groupby("tournament_id")["video_id"].nunique().reset_index().sort_values("tournament_id")

,tournament_id,video_id
0,0,641
1,1,343
2,2,452
3,3,549
4,4,133
...,...,...
59,60,13
60,61,27
61,62,17
62,63,8


In [17]:
dfm["is_downloaded"] = np.where(dfm["video_id"].isin(s_downloaded_vids), True, False)

In [20]:
dfx = dfm[dfm["is_downloaded"]]

In [21]:
dfx.groupby("tournament_id")["video_id"].nunique().reset_index().sort_values("tournament_id")

,tournament_id,video_id
0,0,1
1,1,1
2,2,2
3,3,59
4,4,44
...,...,...
59,60,7
60,61,10
61,62,8
62,63,8


### Total videos downloaded

In [3]:
s_downloaded_vids = Utils.list_videos(PATH_VIDEOS)

In [4]:
len(s_downloaded_vids)

1220

In [ ]:


# ---------------------------------------------------- #
#     Concatenate Tennis TV and Grandslams videos
# ---------------------------------------------------- #
df_gs = pd.read_parquet(PATH_MASTER_GS, engine="pyarrow")
df_tennistv_videos = pd.read_parquet(PATH_MASTER_TENNIS_TV, engine="pyarrow")
df_base = pd.concat([df_tennistv_videos, df_gs])